# Crime Reporting Assessment

**Database file:** `crime_reports.db`  
**Purpose:** Explore vehicle theft reports with SQL. Use `%%sql` cell magic (ipython-sql) to run queries directly against the SQLite database.

**Quick setup:**
1. Install dependencies (if needed):  
```bash
pip install ipython-sql sqlalchemy
```
2. In a notebook cell, load the extension:
```python
%load_ext sql
```
3. Connect using cell magic for SQLite (the notebook assumes `crime_reports.db` is in the same folder):
```sql
%%sql sqlite:///crime_reports.db
SELECT name FROM sqlite_master WHERE type='table';
```

Below are 10 questions. Each question has:
- A short description
- A `%%sql` cell that students should run
- A short note on what to look for in the results

---



### Q1 — Count number of cars per individual and show top 10 owners
Use `GROUP BY` to aggregate cars per owner, order by count desc.

In [ ]:
%%sql sqlite:///crime_reports.db
SELECT i.id AS individual_id,
       i.first_name || ' ' || i.last_name AS owner,
       COUNT(c.id) AS cars_owned
FROM individuals i
LEFT JOIN cars c ON c.individual_id = i.id
GROUP BY i.id
HAVING cars_owned > 0
ORDER BY cars_owned DESC
LIMIT 10;


### Q2 — List individuals who own more than 3 cars (use HAVING)
Practice `HAVING` to filter aggregated results.

In [ ]:
%%sql sqlite:///crime_reports.db
SELECT i.id AS individual_id,
       i.first_name || ' ' || i.last_name AS owner,
       COUNT(c.id) AS cars_owned
FROM individuals i
LEFT JOIN cars c ON c.individual_id = i.id
GROUP BY i.id
HAVING COUNT(c.id) > 3
ORDER BY cars_owned DESC;


### Q3 — Get recent reports with car, owner and report status (example of JOINs)
Join reports -> cars -> individuals -> locations to get full context.

In [ ]:
%%sql sqlite:///crime_reports.db
SELECT r.id AS report_id,
       r.reported_at,
       r.status,
       c.plate,
       c.make || ' ' || c.model AS car,
       i.first_name || ' ' || i.last_name AS owner,
       loc.name AS last_seen_location
FROM reports r
JOIN cars c ON r.car_id = c.id
JOIN individuals i ON c.individual_id = i.id
JOIN locations loc ON r.last_seen_location_id = loc.id
ORDER BY r.reported_at DESC
LIMIT 20;


### Q4 — Count reports per status using a subquery
Use a subquery to compute totals per status.

In [ ]:
%%sql sqlite:///crime_reports.db
SELECT status, COUNT(*) AS reports_count
FROM reports
GROUP BY status
ORDER BY reports_count DESC;


### Q5 — Investigator workload (number of reports assigned) using a CTE
Use a CTE to compute counts and then join to investigator info.

In [ ]:
%%sql sqlite:///crime_reports.db
WITH inv_counts AS (
    SELECT investigator_id, COUNT(*) AS assigned_reports
    FROM reports
    WHERE investigator_id IS NOT NULL
    GROUP BY investigator_id
)
SELECT inv.id, inv.first_name || ' ' || inv.last_name AS investigator, ic.assigned_reports, loc.name AS jurisdiction
FROM inv_counts ic
JOIN investigators inv ON inv.id = ic.investigator_id
LEFT JOIN locations loc ON inv.jurisdiction_location_id = loc.id
ORDER BY ic.assigned_reports DESC
LIMIT 15;


### Q6 — Latest report per car using ROW_NUMBER() window function
Use a window function to pick the most recent report per car.

In [ ]:
%%sql sqlite:///crime_reports.db
WITH ranked_reports AS (
    SELECT r.*,
           ROW_NUMBER() OVER (PARTITION BY car_id ORDER BY reported_at DESC) AS rn
    FROM reports r
)
SELECT rr.id AS report_id, rr.car_id, rr.reported_at, rr.status
FROM ranked_reports rr
WHERE rr.rn = 1
ORDER BY rr.reported_at DESC
LIMIT 50;


### Q7 — Average distance from owner by report status (joins + aggregate)
Find which statuses have higher average distance from owner.

In [ ]:
%%sql sqlite:///crime_reports.db
SELECT status, ROUND(AVG(distance_from_owner_km),2) AS avg_distance_km, COUNT(*) AS reports_count
FROM reports
GROUP BY status
HAVING COUNT(*) > 5
ORDER BY avg_distance_km DESC;


### Q8 — Owners with cars reported more than once (subquery)
Use a subquery to find cars with multiple reports then list owners.

In [ ]:
%%sql sqlite:///crime_reports.db
SELECT DISTINCT i.id AS owner_id, i.first_name || ' ' || i.last_name AS owner, COUNT(r.id) AS total_reports
FROM individuals i
JOIN cars c ON c.individual_id = i.id
JOIN reports r ON r.car_id = c.id
GROUP BY i.id
HAVING total_reports > 1
ORDER BY total_reports DESC
LIMIT 20;


### Q9 — Top 3 most reported cars per last-seen location (CTE + ROW_NUMBER window)
Use CTE and windowing to rank cars per location.

In [ ]:
%%sql sqlite:///crime_reports.db
WITH car_reports AS (
    SELECT r.last_seen_location_id, c.id AS car_id, c.plate, COUNT(r.id) AS reports_count
    FROM reports r
    JOIN cars c ON r.car_id = c.id
    GROUP BY r.last_seen_location_id, c.id
), ranked AS (
    SELECT cr.*, ROW_NUMBER() OVER (PARTITION BY last_seen_location_id ORDER BY reports_count DESC) AS rn
    FROM car_reports cr
)
SELECT loc.name AS location, ranked.plate, ranked.reports_count
FROM ranked
JOIN locations loc ON loc.id = ranked.last_seen_location_id
WHERE ranked.rn <= 3
ORDER BY loc.name, ranked.reports_count DESC;


### Q10 — First (earliest) report assigned to each investigator by their jurisdiction (ROW_NUMBER)
Demonstrates partitioning by investigator and ordering by date.

In [ ]:
%%sql sqlite:///crime_reports.db
WITH inv_reports AS (
    SELECT inv.id AS investigator_id, inv.first_name || ' ' || inv.last_name AS investigator,
           r.id AS report_id, r.reported_at, inv.jurisdiction_location_id,
           ROW_NUMBER() OVER (PARTITION BY inv.id ORDER BY r.reported_at ASC) AS rn
    FROM investigators inv
    JOIN reports r ON r.investigator_id = inv.id
)
SELECT ir.investigator_id, ir.investigator, ir.report_id, ir.reported_at, loc.name AS jurisdiction
FROM inv_reports ir
LEFT JOIN locations loc ON loc.id = ir.jurisdiction_location_id
WHERE ir.rn = 1
ORDER BY ir.investigator;


---
## Tips for grading with NBGrader
- NBGrader can capture cell outputs; ensure assignments expect the resulting table or scalar value.
- You can make a hidden test cell that runs the same SQL and checks results programmatically using Python and sqlite3.
- Remind students to keep the DB and notebook in the same directory, or adjust the connection string accordingly.
